In [10]:
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import OneHotEncoder

def create_gold_dataset(glycans_file, proteins_file, train_fractions_file, output_file):
    """
    Create a gold dataset by merging glycans, proteins, and train_fractions data.
    Performs one-hot encoding on categorical features.
    
    Parameters:
    -----------
    glycans_file : str
        Path to the detailed_glycans CSV file
    proteins_file : str
        Path to the detailed_proteins CSV file
    train_fractions_file : str
        Path to the train_fractions CSV file
    output_file : str
        Path to save the output gold_data CSV file
    
    Returns:
    --------
    pandas.DataFrame
        The merged gold dataset with one-hot encoded categorical features
    """
    print(f"Reading glycans data from {glycans_file}...")
    glycans = pd.read_csv(glycans_file)
    print(f"Read {len(glycans)} glycan records")
    
    print(f"Reading proteins data from {proteins_file}...")
    proteins = pd.read_csv(proteins_file)
    print(f"Read {len(proteins)} protein records")
    
    print(f"Reading train fractions data from {train_fractions_file}...")
    train_fractions = pd.read_csv(train_fractions_file, sep='\t')
    print(f"Read {len(train_fractions)} train fraction records")
    
    # Create maps for faster lookup
    glycans_map = {row['Name']: row for _, row in glycans.iterrows()}
    proteins_map = {row['ProteinGroup']: row for _, row in proteins.iterrows()}
    
    # Create the gold dataset by joining the data
    print("Merging datasets...")
    gold_data_records = []
    
    # Track how many records were successfully merged and how many had missing data
    total_records = len(train_fractions)
    missing_glycan = 0
    missing_protein = 0
    successful_merges = 0
    
    for _, fraction in train_fractions.iterrows():
        glycan_id = fraction['GlycanID']
        protein_group = fraction['ProteinGroup']
        
        # Get the corresponding glycan and protein records
        glycan = glycans_map.get(glycan_id)
        protein = proteins_map.get(protein_group)
        
        # Skip if either glycan or protein not found
        if glycan is None:
            missing_glycan += 1
            continue
            
        if protein is None:
            missing_protein += 1
            continue
        
        # Create a merged record
        merged_record = {
            # Base fields from train_fractions
            'ObjId': fraction['ObjId'],
            'ProteinGroup': fraction['ProteinGroup'],
            'Concentration': fraction['Concentration'],
            'GlycanID': fraction['GlycanID'],
            'f': fraction['f']
        }
        
        # Add prefix to glycan columns to avoid name conflicts
        for key, value in glycan.items():
            merged_record[f'Glycan_{key}'] = value
            
        # Add prefix to protein columns to avoid name conflicts
        for key, value in protein.items():
            merged_record[f'Protein_{key}'] = value
        
        gold_data_records.append(merged_record)
        successful_merges += 1
    
    # Create DataFrame from the merged records
    gold_data = pd.DataFrame(gold_data_records)

    # Remove specified columns
    columns_to_remove = ['GlycanID', 'Glycan_Name', 'Glycan_IUPAC', 'Glycan_SMILES', 'Glycan_GlycanClass', # Category columns
                         'Glycan_SlogP_VSA9', 'Glycan_SMR_VSA8', 'Glycan_PyranoseCount', 'Glycan_FuranoseCount', 'Glycan_BranchingPoints'] # numerical columns
    gold_data.drop(columns=columns_to_remove, inplace=True)
    
    # Print summary statistics
    print("\nMerge Summary:")
    print(f"Total train_fractions records: {total_records}")
    print(f"Records with missing glycans: {missing_glycan}")
    print(f"Records with missing proteins: {missing_protein}")
    print(f"Successfully merged records: {successful_merges}")
    
    # Perform one-hot encoding on categorical features
    print("\nPerforming one-hot encoding on categorical features...")

    
    # Combine all categorical columns
    categorical_columns = gold_data.select_dtypes(include=['object']).columns
    
    # Check if these columns exist in the dataframe
    existing_categorical_columns = [col for col in categorical_columns if col in gold_data.columns]
    
    if existing_categorical_columns:
        # Create a copy to preserve original data
        encoded_gold_data = gold_data.copy()
        
        # Handle each categorical column
        total_encoded_features = 0
        
        for col in existing_categorical_columns:
            # Fill NaN values with a placeholder
            encoded_gold_data[col] = encoded_gold_data[col].fillna('Unknown')
            
            # Get unique values for this categorical column
            unique_values = encoded_gold_data[col].unique()
            print(f"One-hot encoding {col} with {len(unique_values)} unique values")
            
            # Skip one-hot encoding if there's only one unique value
            if len(unique_values) <= 1:
                print(f"  Skipping {col} as it has only {len(unique_values)} unique value(s)")
                continue
                
            # Create one-hot encoder
            encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
            
            # Fit and transform
            encoded_values = encoder.fit_transform(encoded_gold_data[[col]])
            
            # Create new column names for encoded features
            feature_names = [f"{col}[{val}]" for val in encoder.categories_[0]]
            
            # Create a DataFrame with the encoded values
            encoded_df = pd.DataFrame(encoded_values, columns=feature_names, index=encoded_gold_data.index)
            
            # Concatenate the encoded values with the original DataFrame
            encoded_gold_data = pd.concat([encoded_gold_data, encoded_df], axis=1)
            
            # Remove the original categorical column
            encoded_gold_data.drop(col, axis=1, inplace=True)
            
            total_encoded_features += len(feature_names)
        
        # Update gold_data to the encoded version
        gold_data = encoded_gold_data
        print(f"One-hot encoding added {total_encoded_features} new features")
    else:
        print("No categorical columns found in the data. Skipping one-hot encoding.")
    
    # Save the gold dataset to CSV
    print(f"\nSaving gold dataset to {output_file}...")
    gold_data.to_csv(output_file, index=False)
    print(f"Gold dataset saved with {len(gold_data)} records and {len(gold_data.columns)} columns")
    
    return gold_data


In [11]:
# Define file paths
glycans_file = "detailed_glycans.csv"  
proteins_file = "detailed_proteins.csv"  
train_fractions_file = "Train_Fractions.csv"
output_file = "gold_data.csv"

# Create the gold dataset
gold_data = create_gold_dataset(glycans_file, proteins_file, train_fractions_file, output_file)

# Display a sample of the gold dataset
print("\nSample of gold dataset:")
print(gold_data.head(3))

# Display the column names
print("\nGold dataset columns:")
for i, col in enumerate(gold_data.columns):
    print(f"{i+1}. {col}")

Reading glycans data from detailed_glycans.csv...
Read 611 glycan records
Reading proteins data from detailed_proteins.csv...
Read 52 protein records
Reading train fractions data from Train_Fractions.csv...
Read 68492 train fraction records
Merging datasets...

Merge Summary:
Total train_fractions records: 68492
Records with missing glycans: 0
Records with missing proteins: 0
Successfully merged records: 68492

Performing one-hot encoding on categorical features...
One-hot encoding Protein_Accession with 37 unique values
One-hot encoding Protein_Uniprot with 37 unique values
One-hot encoding Protein_Description with 26 unique values
One-hot encoding Protein_Amino Acid Sequence with 37 unique values
One-hot encoding Protein_ProteinID with 37 unique values
One-hot encoding Protein_CleanSequence with 37 unique values
One-hot encoding Protein_PredictedSecondaryStructure with 3 unique values
One-hot encoding Protein_Stability with 2 unique values
One-hot encoding Protein_PredictedProteinCla

In [12]:
gold_data

,ObjId,ProteinGroup,Concentration,f,Glycan_MolecularWeight,Glycan_HeavyAtomCount,Glycan_RingCount,Glycan_RotatableBondCount,Glycan_HBondDonorCount,Glycan_HBondAcceptorCount,...,Protein_PredictedSecondaryStructure[Turn],Protein_Stability[Stable],Protein_Stability[Unstable],Protein_PredictedProteinClass[Acidic Protein],Protein_PredictedProteinClass[Aromatic-rich Protein],Protein_PredictedProteinClass[Beta-sheet Protein],Protein_PredictedProteinClass[Disulfide-rich Protein],Protein_PredictedProteinClass[Globular Protein],Protein_PredictedProteinClass[Highly Stable Protein],Protein_PredictedProteinClass[Membrane Protein]
0,1004699,1,0.001,0.000154,337.369,23,1,15,5,8,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1004699,1,0.001,0.000082,337.369,23,1,15,5,8,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1004699,1,0.001,0.000290,408.404,28,1,16,6,10,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1004699,1,0.001,0.000000,378.422,26,1,16,5,8,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1004699,1,0.001,0.000000,321.370,22,1,14,4,7,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68487,1006330,146,200.000,0.000401,1079.022,74,4,41,17,28,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
68488,1006330,146,200.000,0.047294,3899.569,267,19,141,65,106,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
68489,1006330,146,200.000,0.003314,3168.899,217,15,115,53,86,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
68490,1006330,146,200.000,0.001464,539.579,37,2,22,7,13,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
gold_data.shape

(68492, 145)

In [ ]:
# Select categorical columns
categorical_columns = gold_data.select_dtypes(include=['object']).columns

# Get the number of unique values for each categorical column
unique_values = {col: gold_data[col].nunique() for col in categorical_columns}

# Display the result
unique_values

{'Protein_Accession': 37,
 'Protein_Uniprot': 37,
 'Protein_Description': 26,
 'Protein_Amino Acid Sequence': 37,
 'Protein_ProteinID': 37,
 'Protein_CleanSequence': 37,
 'Protein_PredictedSecondaryStructure': 3,
 'Protein_Stability': 2,
 'Protein_PredictedProteinClass': 7}

In [ ]:
# Select numerical columns
numerical_columns = gold_data.select_dtypes(include=['number']).columns

# Get the number of unique values for each numerical column
unique_values_numerical = {col: gold_data[col].nunique() for col in numerical_columns}

# Display the result
{col: count for col, count in unique_values_numerical.items() if count == 1}

{}

In [ ]:
# Get value counts for each categorical column with less than 100 unique values
value_counts = {col: gold_data[col].value_counts() for col in categorical_columns if unique_values[col] < 100}

# Display the result
for col, counts in value_counts.items():
    print(f"Value counts for {col}:")
    print(counts)
    print("\n")

Value counts for Protein_Accession:
Protein_Accession
Q41358    4167
A8WDZ4    2778
P18674    2315
P10968    2315
Q71QF2    1852
B2ZRS9    1852
P30617    1852
P05045    1852
P29257    1852
P02870    1852
G9M5T0    1852
C0HM45    1852
D0VWW1    1852
P05088    1852
B5A8N6    1852
Q5ZET2    1852
O24313    1852
P06750    1852
P02872    1852
Q9S8M0    1852
P11218    1852
P22973    1852
Q05097    1852
F6KMV5    1848
Q7MDS1    1848
Q8L5H4    1848
H2FH31    1848
P47929    1848
E0UGD2    1389
Q2UNX8    1389
Q00022    1389
P16030    1389
Q7Y041    1389
Q8X123    1389
P84871    1386
B3VS76    1386
P58908    1383
Name: count, dtype: int64


Value counts for Protein_Uniprot:
Protein_Uniprot
SNAIB_SAMNI     4167
A8WDZ4_CANEN    2778
LECA_MACPO      2315
AGI1_WHEAT      2315
Q71QF2_AMACA    1852
CNL_CLINE       1852
LEC_GALNI       1852
LEC1_VIGUC      1852
LEC2_CYTSC      1852
LEC_LENCU       1852
G9M5T0_SOLLC    1852
LEC_NARPS       1852
D0VWW1_LOTTE    1852
PHAE_PHAVU      1852
B5A8N6_PEA      185